In [ ]:
# ================================================================================
# NFL BIG DATA BOWL 2026 - IMPROVED SOLUTION
# Enhanced player movement prediction with advanced temporal features and physics
# ================================================================================

# Imports
import os
import gc
import math
import random
from pathlib import Path

import numpy as np
import pandas as pd

import warnings
warnings.filterwarnings('ignore')

from tqdm.auto import tqdm
from scipy.ndimage import gaussian_filter1d

from sklearn.model_selection import GroupKFold
from sklearn.preprocessing import StandardScaler

from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset

In [ ]:
# ================================================================================
# CONFIGURATION
# ================================================================================

class Config:
    # Data paths
    DATA_DIR = Path("/kaggle/input/nfl-big-data-bowl-2026-prediction/")

    # Random seeds
    SEED = 42
    SEEDS = [42, 123, 2024]

    # Cross-validation
    N_FOLDS = 5

    # Field geometry
    FIELD_X_MIN = 0.0
    FIELD_X_MAX = 120.0
    FIELD_Y_MIN = 0.0
    FIELD_Y_MAX = 53.3

    # Physical constraints
    MAX_SPEED = 12.0

    # Neural net training
    NN_BATCH_SIZE = 2048
    NN_EPOCHS = 100
    NN_LEARNING_RATE = 5e-4
    NN_PATIENCE = 10

    # GPU preferences
    USE_GPU = True
    GPU_DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
    GPU_COUNT = torch.cuda.device_count() if torch.cuda.is_available() else 0

    # Early stopping for tree models
    ES_ROUNDS = 200

    # Verbosity for third-party libs
    VERBOSE = False

    # Weeks expected
    WEEKS = list(range(1, 18))

    # File names
    TEST_INPUT = "test_input.csv"
    TEST_TEMPLATE = "test.csv"

    @classmethod
    def print_gpu_info(cls):
        # Header
        print("=" * 60)
        print("GPU CONFIGURATION")
        print("=" * 60)

        # CUDA availability
        print(f"CUDA Available: {torch.cuda.is_available()}")

        # Device and count
        print(f"GPU Device: {cls.GPU_DEVICE}")
        print(f"GPU Count: {cls.GPU_COUNT}")

        # Detailed GPU listing
        if torch.cuda.is_available():
            for i in range(cls.GPU_COUNT):
                name = torch.cuda.get_device_name(i)
                mem_gb = torch.cuda.get_device_properties(i).total_memory / 1024**3
                print(f"GPU {i}: {name} ({mem_gb:.1f} GB)")
        else:
            print("No GPU available - using CPU")

        # Footer
        print("=" * 60)

    @classmethod
    def cleanup_gpu_memory(cls):
        # Clear CUDA cache
        if torch.cuda.is_available():
            torch.cuda.empty_cache()
            torch.cuda.synchronize()

        # Run GC
        gc.collect()

In [ ]:
# ================================================================================
# UTILS
# ================================================================================

def set_all_seeds(seed):
    # Python RNG
    random.seed(seed)

    # Numpy RNG
    np.random.seed(seed)

    # Torch RNG
    torch.manual_seed(seed)

    # CUDA RNG
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)


def check_gpu_requirements():
    # CUDA availability
    gpu_available = torch.cuda.is_available()

    # CUDA status
    if gpu_available:
        print("✅ CUDA is available")
    else:
        print("⚠️ CUDA not available - will use CPU for all models")

    # XGBoost availability
    try:
        import xgboost  # noqa
        print("✅ XGBoost installed")
    except Exception:
        print("⚠️ XGBoost not found - install with: pip install xgboost")

    # LightGBM availability
    try:
        import lightgbm  # noqa
        print("✅ LightGBM installed")
    except Exception:
        print("⚠️ LightGBM not found - install with: pip install lightgbm")

    # CatBoost availability
    try:
        import catboost  # noqa
        print("✅ CatBoost installed")
    except Exception:
        print("⚠️ CatBoost not found - install with: pip install catboost")

    # Return status
    return gpu_available


def to_inches(height_str):
    # Handle non-string
    if not isinstance(height_str, str):
        return 70

    # Handle malformed
    if '-' not in height_str:
        return 70

    # Convert ft-in to inches
    try:
        feet, inches = map(int, height_str.split('-'))
        return feet * 12 + inches
    except Exception:
        return 70


def safe_cut(series, bins, labels=False):
    # Robust pd.cut with NaN handling
    try:
        return pd.cut(series, bins=bins, labels=labels)
    except Exception:
        return pd.Series([np.nan] * len(series), index=series.index)

In [ ]:
# ================================================================================
# SIMPLE NEURAL NETWORK
# ================================================================================

class SimpleNN(nn.Module):
    # Constructor
    def __init__(self, input_dim):
        super().__init__()

        # Layers
        self.layers = nn.Sequential(
            nn.Linear(input_dim, 256),
            nn.ReLU(),
            nn.Dropout(0.30),
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.Dropout(0.20),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Linear(64, 1),
        )

    # Forward pass
    def forward(self, x):
        return self.layers(x)

In [ ]:
# ================================================================================
# ENCODER WITH UNSEEN HANDLING
# ================================================================================

class SafeLabelEncoder:
    # Constructor
    def __init__(self):
        self.classes_ = {}
        self.fitted = False

    # Fit on training column
    def fit(self, series):
        # Fill NaNs
        ser = series.fillna('Unknown').astype(str)

        # Build classes
        uniq = sorted(ser.unique().tolist())

        # Ensure Unknown at index 0
        if 'Unknown' not in uniq:
            uniq = ['Unknown'] + uniq

        # Map
        self.classes_ = {c: i for i, c in enumerate(uniq)}

        # Flag
        self.fitted = True

        # Return self
        return self

    # Transform any column
    def transform(self, series):
        # Precondition
        if not self.fitted:
            raise RuntimeError("SafeLabelEncoder not fitted")

        # Fill NaNs
        ser = series.fillna('Unknown').astype(str)

        # Map with Unknown for unseen
        return ser.map(lambda x: self.classes_.get(x, self.classes_['Unknown'])).astype(int)

In [ ]:
# ================================================================================
# MAIN PIPELINE
# ================================================================================

class EnhancedNFLPlayerMovementPredictor:
    # Constructor
    def __init__(self, data_dir, seed=42):
        # Paths and seed
        self.data_dir = Path(data_dir)
        self.seed = seed

        # CV holders
        self.models_dx = {'xgb': [], 'lgb': [], 'cat': []}
        self.models_dy = {'xgb': [], 'lgb': [], 'cat': []}
        self.nn_models_dx = []
        self.nn_models_dy = []

        # Scalers and encoders
        self.num_scaler = StandardScaler()
        self.encoders = {}

        # Feature lists
        self.numerical_features = []
        self.categorical_features = []

        # Dataframes
        self.train_data = None
        self.test_data = None

    # Load many CSVs
    def _load_many(self, paths):
        # Container
        frames = []

        # Loop
        for p in tqdm(paths, desc="Loading files"):
            try:
                frames.append(pd.read_csv(p))
            except Exception:
                pass

        # Concatenate
        if len(frames) == 0:
            return pd.DataFrame()

        # Return concat
        return pd.concat(frames, ignore_index=True)

    # Load all datasets
    def load_and_combine_datasets(self):
        # Compose input paths
        inputs = [self.data_dir / f"train/input_2023_w{w:02d}.csv" for w in Config.WEEKS]

        # Compose output paths
        outputs = [self.data_dir / f"train/output_2023_w{w:02d}.csv" for w in Config.WEEKS]

        # Filter existence
        inputs = [p for p in inputs if p.exists()]
        outputs = [p for p in outputs if p.exists()]

        # Logs
        print(f"Found {len(inputs)} weeks of input and {len(outputs)} weeks of output")

        # Read train input/output
        train_input = self._load_many(inputs)
        train_output = self._load_many(outputs)

        # Read test input/template
        test_input = pd.read_csv(self.data_dir / Config.TEST_INPUT)
        test_template = pd.read_csv(self.data_dir / Config.TEST_TEMPLATE)

        # Log sizes
        print(f"Loaded train_input: {len(train_input):,} rows")
        print(f"Loaded train_output: {len(train_output):,} rows")
        print(f"Loaded test_input:  {len(test_input):,} rows")
        print(f"Loaded test_template: {len(test_template):,} rows")

        # Return
        return train_input, train_output, test_input, test_template

    # Temporal features from tracking
    def _extract_temporal_features(self, tracking):
        # Sort and take last frame
        last = tracking.sort_values(['game_id', 'play_id', 'nfl_id', 'frame_id']).groupby(
            ['game_id', 'play_id', 'nfl_id'], as_index=False
        ).last()

        # Rename last frame columns
        last = last.rename(columns={'x': 'final_pre_throw_x', 'y': 'final_pre_throw_y'})

        # Aggregate statistics
        agg = tracking.groupby(['game_id', 'play_id', 'nfl_id']).agg({
            'x': ['mean', 'std', 'min', 'max'],
            'y': ['mean', 'std', 'min', 'max'],
            's': ['mean', 'std', 'max', 'min'],
            'a': ['mean', 'std', 'max', 'min'],
            'dir': lambda x: np.std(np.diff(x)) if len(x) > 1 else 0,
            'o':   lambda x: np.std(np.diff(x)) if len(x) > 1 else 0,
        }).reset_index()

        # Flatten columns
        agg.columns = ['_'.join(c).strip('_') for c in agg.columns.to_flat_index()]

        # Rename lambdas
        agg = agg.rename(columns={
            'dir_<lambda_0>': 'dir_change_rate',
            'o_<lambda_0>': 'orientation_change_rate'
        })

        # Last N frames
        N = 5

        # Take recent frames
        recent = tracking.sort_values(['game_id', 'play_id', 'nfl_id', 'frame_id']).groupby(
            ['game_id', 'play_id', 'nfl_id']
        ).tail(N)

        # Trajectory deltas
        traj = recent.groupby(['game_id', 'play_id', 'nfl_id']).agg({
            'x': lambda x: (x.iloc[-1] - x.iloc[0]) if len(x) > 1 else 0,
            'y': lambda x: (x.iloc[-1] - x.iloc[0]) if len(x) > 1 else 0,
            's': lambda x: x.diff().mean() if len(x) > 1 else 0,
        }).reset_index()

        # Rename trajectory columns
        traj.columns = ['game_id', 'play_id', 'nfl_id',
                        'recent_displacement_x', 'recent_displacement_y', 'acceleration_trend']

        # Merge last + agg + traj
        out = last.merge(agg, on=['game_id', 'play_id', 'nfl_id'], how='left')
        out = out.merge(traj, on=['game_id', 'play_id', 'nfl_id'], how='left')

        # Convert height if present
        if 'player_height' in out.columns:
            out['height_inches'] = out['player_height'].apply(to_inches)

        # Return features
        return out

    # Merge targeted receiver coordinates
    def _incorporate_target_receiver_data(self, df):
        # Column guard
        if 'player_role' not in df.columns:
            out = df.copy()
            out['target_receiver_x'] = np.nan
            out['target_receiver_y'] = np.nan
            return out

        # Extract target receiver per play
        trg = df[df['player_role'] == "Targeted Receiver"][
            ['game_id', 'play_id', 'final_pre_throw_x', 'final_pre_throw_y']
        ].drop_duplicates(['game_id', 'play_id'])

        # Rename target columns
        trg = trg.rename(columns={
            'final_pre_throw_x': 'target_receiver_x',
            'final_pre_throw_y': 'target_receiver_y'
        })

        # Merge back
        return df.merge(trg, on=['game_id', 'play_id'], how='left')

    # Advanced features
    def _calculate_advanced_features(self, df_in, training_mode=False):
        # Copy
        df = df_in.copy()

        # Time features
        if 'frame_id' in df.columns:
            df['time_seconds'] = df['frame_id'] / 10.0
            df['time_normalized'] = df['frame_id'] / df.groupby(
                ['game_id', 'play_id', 'nfl_id']
            )['frame_id'].transform('max')
            df['time_squared'] = df['time_seconds'] ** 2
            df['time_cubed'] = df['time_seconds'] ** 3
            df['sqrt_time'] = np.sqrt(np.clip(df['time_seconds'], 0, None))
            df['log_time'] = np.log1p(np.clip(df['time_seconds'], 0, None))
            df['time_sin'] = np.sin(2 * np.pi * df['time_normalized'])
            df['time_cos'] = np.cos(2 * np.pi * df['time_normalized'])
            df['time_sin_2'] = np.sin(4 * np.pi * df['time_normalized'])
            df['time_cos_2'] = np.cos(4 * np.pi * df['time_normalized'])
            df['is_early_play'] = (df['time_normalized'] < 0.33).astype(int)
            df['is_mid_play'] = ((df['time_normalized'] >= 0.33) & (df['time_normalized'] < 0.67)).astype(int)
            df['is_late_play'] = (df['time_normalized'] >= 0.67).astype(int)

        # Velocity and physics
        if {'s', 'dir'}.issubset(df.columns):
            rad = np.deg2rad(df['dir'].fillna(0))
            df['velocity_x'] = df['s'] * np.sin(rad)
            df['velocity_y'] = df['s'] * np.cos(rad)

        # Momentum
        if {'player_weight', 's'}.issubset(df.columns):
            df['momentum_magnitude'] = df['player_weight'] * df['s']

        # Constant velocity expectation
        if {'time_seconds', 'final_pre_throw_x', 'final_pre_throw_y', 'velocity_x', 'velocity_y'}.issubset(df.columns):
            df['expected_x_constant_v'] = df['final_pre_throw_x'] + df['velocity_x'] * df['time_seconds']
            df['expected_y_constant_v'] = df['final_pre_throw_y'] + df['velocity_y'] * df['time_seconds']

        # With acceleration
        if {'time_seconds', 'time_squared', 'a', 'dir', 'final_pre_throw_x', 'final_pre_throw_y',
             'velocity_x', 'velocity_y'}.issubset(df.columns):
            rad = np.deg2rad(df['dir'].fillna(0))
            df['expected_x_with_accel'] = (
                df['final_pre_throw_x'] + df['velocity_x'] * df['time_seconds'] + 0.5 * df['a'] * np.sin(rad) * df['time_squared']
            )
            df['expected_y_with_accel'] = (
                df['final_pre_throw_y'] + df['velocity_y'] * df['time_seconds'] + 0.5 * df['a'] * np.cos(rad) * df['time_squared']
            )

        # Speed/accel consistency
        if {'s', 's_mean'}.issubset(df.columns):
            df['speed_consistency'] = df['s'] / (df['s_mean'] + 0.1)
            df['speed_deviation'] = np.abs(df['s'] - df['s_mean'])

        if {'a', 'a_mean'}.issubset(df.columns):
            df['acceleration_consistency'] = df['a'] / (df['a_mean'] + 0.1)
            df['acceleration_deviation'] = np.abs(df['a'] - df['a_mean'])

        # Time interactions
        if {'time_seconds', 's'}.issubset(df.columns):
            df['time_x_speed'] = df['time_seconds'] * df['s']

        if {'time_seconds', 'a'}.issubset(df.columns):
            df['time_x_acceleration'] = df['time_seconds'] * df['a']

        if {'time_squared', 's'}.issubset(df.columns):
            df['time_squared_x_speed'] = df['time_squared'] * df['s']

        # Ball trajectory features
        if {'ball_land_x', 'ball_land_y', 'final_pre_throw_x', 'final_pre_throw_y'}.issubset(df.columns):
            dx = df['ball_land_x'] - df['final_pre_throw_x']
            dy = df['ball_land_y'] - df['final_pre_throw_y']
            dist = np.sqrt(dx ** 2 + dy ** 2)
            df['distance_to_ball_landing'] = dist
            df['angle_to_ball_landing'] = np.arctan2(dy, dx)
            df['ball_direction_x'] = dx / (dist + 1e-6)
            df['ball_direction_y'] = dy / (dist + 1e-6)
            df['estimated_time_to_ball'] = dist / 20.0
            if 'time_seconds' in df.columns:
                df['time_ratio_to_ball'] = df['time_seconds'] / (df['estimated_time_to_ball'] + 0.1)
            if {'velocity_x', 'velocity_y'}.issubset(df.columns):
                bux = dx / (dist + 1e-6)
                buy = dy / (dist + 1e-6)
                df['closing_speed'] = df['velocity_x'] * bux + df['velocity_y'] * buy
                df['projected_time_to_ball'] = dist / (np.abs(df['closing_speed']) + 0.1)
                if 'time_seconds' in df.columns:
                    df['time_urgency'] = df['time_seconds'] / (df['projected_time_to_ball'] + 0.1)
            if 'time_seconds' in df.columns:
                df['distance_to_ball_x_time'] = dist * df['time_seconds']
            if 'time_squared' in df.columns:
                df['distance_to_ball_x_time_squared'] = dist * df['time_squared']

        # Target receiver features
        if {'target_receiver_x', 'target_receiver_y', 'final_pre_throw_x', 'final_pre_throw_y'}.issubset(df.columns):
            tdx = df['target_receiver_x'] - df['final_pre_throw_x']
            tdy = df['target_receiver_y'] - df['final_pre_throw_y']
            tdist = np.sqrt(tdx ** 2 + tdy ** 2)
            df['distance_to_target'] = tdist
            df['angle_to_target'] = np.arctan2(tdy, tdx)
            if 'time_seconds' in df.columns:
                df['distance_to_target_x_time'] = tdist * df['time_seconds']

        # Target indicator
        if 'player_role' in df.columns:
            df['is_target_receiver'] = (df['player_role'] == "Targeted Receiver").astype(int)
        else:
            df['is_target_receiver'] = 0

        # Field position
        if 'final_pre_throw_x' in df.columns:
            df['normalized_x'] = df['final_pre_throw_x'] / Config.FIELD_X_MAX
            df['field_region_x'] = safe_cut(df['final_pre_throw_x'], bins=6, labels=False)
            df['distance_from_endzone'] = np.minimum(
                df['final_pre_throw_x'], Config.FIELD_X_MAX - df['final_pre_throw_x']
            )

        if 'final_pre_throw_y' in df.columns:
            df['normalized_y'] = df['final_pre_throw_y'] / Config.FIELD_Y_MAX
            df['field_region_y'] = safe_cut(df['final_pre_throw_y'], bins=4, labels=False)
            df['distance_from_sideline'] = np.minimum(
                df['final_pre_throw_y'], Config.FIELD_Y_MAX - df['final_pre_throw_y']
            )

        # Game context
        if 'absolute_yardline_number' in df.columns:
            df['yards_to_endzone'] = df['absolute_yardline_number']
            df['is_redzone'] = (df['absolute_yardline_number'] <= 20).astype(int)

        # Team indicators
        if 'player_side' in df.columns:
            df['is_offense'] = (df['player_side'] == 'Offense').astype(int)
        else:
            df['is_offense'] = 0

        if 'player_role' in df.columns:
            df['is_passer'] = (df['player_role'] == 'Passer').astype(int)
            df['is_coverage'] = (df['player_role'] == 'Defensive Coverage').astype(int)
        else:
            df['is_passer'] = 0
            df['is_coverage'] = 0

        # Player attributes
        if {'player_weight', 'height_inches'}.issubset(df.columns):
            ok_h = df['height_inches'] > 0
            df['bmi'] = np.nan
            df.loc[ok_h, 'bmi'] = (
                (df.loc[ok_h, 'player_weight'] * 0.453592)
                / ((df.loc[ok_h, 'height_inches'] * 0.0254) ** 2)
            )

        # Motion analysis
        if {'dir', 'o'}.issubset(df.columns):
            df['speed_orientation_discrepancy'] = np.abs(df['dir'] - df['o'])

        # Interactions
        if {'s', 'a'}.issubset(df.columns):
            df['speed_times_acceleration'] = df['s'] * df['a']

        if {'distance_to_ball_landing', 's'}.issubset(df.columns):
            df['distance_speed_ratio'] = df['distance_to_ball_landing'] / (df['s'] + 1.0)
            df['distance_ball_x_speed'] = df['distance_to_ball_landing'] * df['s']

        # Advanced interactions
        if {'is_target_receiver', 'time_seconds'}.issubset(df.columns):
            df['is_target_x_time'] = df['is_target_receiver'] * df['time_seconds']
        if {'is_target_receiver', 'time_squared'}.issubset(df.columns):
            df['is_target_x_time_squared'] = df['is_target_receiver'] * df['time_squared']
        if {'is_offense', 'is_early_play'}.issubset(df.columns):
            df['is_offense_x_early_play'] = df['is_offense'] * df['is_early_play']
        if {'is_offense', 'is_late_play'}.issubset(df.columns):
            df['is_offense_x_late_play'] = df['is_offense'] * df['is_late_play']
        if {'is_target_receiver', 'is_late_play'}.issubset(df.columns):
            df['is_target_x_late_play'] = df['is_target_receiver'] * df['is_late_play']

        # Additional previously undefined features
        if {'orientation_change_rate', 'speed_deviation'}.issubset(df.columns):
            df['motion_consistency'] = 1.0 / (1.0 + df['orientation_change_rate'] + df['speed_deviation'])

        if 'distance_to_ball_landing' in df.columns:
            diag = math.hypot(Config.FIELD_X_MAX, Config.FIELD_Y_MAX)
            df['proximity_to_ball_ratio'] = 1.0 - (df['distance_to_ball_landing'] / (diag + 1e-6))

        if {'angle_to_ball_landing'}.issubset(df.columns):
            df['lateral_position_importance'] = np.abs(np.sin(df['angle_to_ball_landing']))

        if {'velocity_x', 'play_direction'}.issubset(df.columns):
            sign = df['play_direction'].fillna('right').map({'right': 1.0, 'left': -1.0}).astype(float)
            df['downfield_progress'] = df['velocity_x'] * sign
        elif 'velocity_x' in df.columns:
            df['downfield_progress'] = df['velocity_x']

        # Training targets
        if training_mode and {'x', 'y', 'final_pre_throw_x', 'final_pre_throw_y'}.issubset(df.columns):
            df['displacement_x'] = df['x'] - df['final_pre_throw_x']
            df['displacement_y'] = df['y'] - df['final_pre_throw_y']

        # Return engineered
        return df

    # Full feature preparation
    def prepare_features(self, in_df, out_df, training_mode=False):
        # Temporal aggregation
        tmp = self._extract_temporal_features(in_df)

        # Target receiver merge
        tmp = self._incorporate_target_receiver_data(tmp)

        # Merge columns
        base_keys = ['game_id', 'play_id', 'nfl_id']
        keep_cols = base_keys + [c for c in tmp.columns if c not in base_keys]

        # Merge output/labels with features
        merged = out_df.merge(tmp[keep_cols], on=base_keys, how='left')

        # Advanced features
        return self._calculate_advanced_features(merged, training_mode=training_mode)

    # Fit encoders
    def _fit_categorical_encoders(self, df, cat_cols):
        # Loop
        for c in cat_cols:
            enc = SafeLabelEncoder()
            enc.fit(df[c] if c in df.columns else pd.Series(['Unknown'] * len(df)))
            self.encoders[c] = enc

    # Transform categoricals
    def _transform_categoricals(self, df, cat_cols):
        # Copy
        out = df.copy()

        # Loop
        for c in cat_cols:
            if c in out.columns:
                out[c] = self.encoders[c].transform(out[c])
            else:
                out[c] = self.encoders[c].transform(pd.Series(['Unknown'] * len(out)))

        # Return
        return out

    # Train ensemble
    def train_models(self):
        # GPU info
        Config.print_gpu_info()

        # Seeds
        set_all_seeds(self.seed)

        # Load datasets
        train_input, train_output, test_input, test_template = self.load_and_combine_datasets()

        # Prepare train/test features
        self.train_data = self.prepare_features(train_input, train_output, training_mode=True)
        self.test_data = self.prepare_features(test_input, test_template, training_mode=False)

        # Feature lists
        all_cols = self.train_data.columns.tolist()

        # Numerical feature candidates
        num_candidates = [
            'final_pre_throw_x', 'final_pre_throw_y', 's', 'a', 'o', 'dir',
            'time_seconds', 'time_normalized', 'time_squared', 'time_cubed', 'sqrt_time', 'log_time',
            'time_sin', 'time_cos', 'time_sin_2', 'time_cos_2', 'is_early_play', 'is_mid_play', 'is_late_play',
            'x_mean', 'x_std', 'x_min', 'x_max', 'y_mean', 'y_std', 'y_min', 'y_max',
            's_mean', 's_std', 's_max', 's_min', 'a_mean', 'a_std', 'a_max', 'a_min',
            'dir_change_rate', 'orientation_change_rate', 'recent_displacement_x', 'recent_displacement_y',
            'acceleration_trend',
            'speed_consistency', 'speed_deviation', 'acceleration_consistency', 'acceleration_deviation',
            'velocity_x', 'velocity_y', 'momentum_magnitude', 'expected_x_constant_v', 'expected_y_constant_v',
            'expected_x_with_accel', 'expected_y_with_accel',
            'distance_to_ball_landing', 'angle_to_ball_landing', 'closing_speed', 'ball_direction_x', 'ball_direction_y',
            'estimated_time_to_ball', 'time_ratio_to_ball', 'projected_time_to_ball', 'time_urgency',
            'distance_to_ball_x_time', 'distance_to_ball_x_time_squared',
            'distance_to_target', 'is_target_receiver', 'angle_to_target', 'distance_to_target_x_time',
            'is_target_x_time_squared',
            'normalized_x', 'normalized_y', 'field_region_x', 'field_region_y',
            'distance_from_sideline', 'distance_from_endzone',
            'yards_to_endzone', 'is_offense', 'is_passer', 'is_coverage', 'is_redzone',
            'height_inches', 'player_weight', 'bmi',
            'speed_orientation_discrepancy',
            'motion_consistency', 'proximity_to_ball_ratio', 'lateral_position_importance', 'downfield_progress',
            'speed_times_acceleration', 'distance_speed_ratio', 'distance_ball_x_speed',
            'time_x_speed', 'time_x_acceleration', 'time_squared_x_speed', 'is_target_x_time',
            'is_offense_x_early_play', 'is_offense_x_late_play', 'is_target_x_late_play'
        ]

        # Categorical feature candidates
        cat_candidates = ['player_role', 'player_side', 'play_direction']

        # Keep present features
        self.numerical_features = [c for c in num_candidates if c in all_cols]
        self.categorical_features = [c for c in cat_candidates if c in all_cols]

        # Target existence check
        if not {'displacement_x', 'displacement_y'}.issubset(self.train_data.columns):
            raise KeyError("Target variables (displacement_x, displacement_y) not found in training data")

        # Assemble X/y
        X = self.train_data[self.numerical_features + self.categorical_features].copy()
        y_dx = self.train_data['displacement_x'].values
        y_dy = self.train_data['displacement_y'].values

        # Fit encoders
        self._fit_categorical_encoders(X, self.categorical_features)

        # Transform categoricals
        X = self._transform_categoricals(X, self.categorical_features)

        # Fill NaNs
        X = X.fillna(0)

        # Fit scaler
        self.num_scaler.fit(X[self.numerical_features])

        # Scale numericals
        X.loc[:, self.numerical_features] = self.num_scaler.transform(X[self.numerical_features])

        # CV groups
        groups = self.train_data['game_id'].values

        # Train with CV
        self._train_with_cv(X, y_dx, y_dy, groups)

        # Return self
        return self

    # Cross-validation training
    def _train_with_cv(self, X, y_dx, y_dy, groups):
        # Splitter
        gkf = GroupKFold(n_splits=Config.N_FOLDS)

        # Loop folds
        for fold, (tr_idx, va_idx) in enumerate(gkf.split(X, groups=groups), start=1):
            # Fold split
            X_tr, X_va = X.iloc[tr_idx], X.iloc[va_idx]
            ydx_tr, ydx_va = y_dx[tr_idx], y_dx[va_idx]
            ydy_tr, ydy_va = y_dy[tr_idx], y_dy[va_idx]

            # Logs
            print(f"Fold {fold}/{Config.N_FOLDS}")

            # XGBoost params
            xgb_params = {
                'n_estimators': 5000,
                'learning_rate': 0.05,
                'max_depth': 8,
                'subsample': 0.8,
                'colsample_bytree': 0.8,
                'random_state': self.seed + fold,
                'verbosity': 0,
                'objective': 'reg:squarederror',
            }

            # XGBoost GPU
            if Config.USE_GPU and torch.cuda.is_available():
                xgb_params.update({'tree_method': 'gpu_hist', 'predictor': 'gpu_predictor', 'gpu_id': 0})
                print("  XGBoost: GPU")
            else:
                xgb_params.update({'tree_method': 'hist'})
                print("  XGBoost: CPU")

            # LightGBM params
            lgb_params = {
                'n_estimators': 5000,
                'learning_rate': 0.05,
                'max_depth': 8,
                'num_leaves': 100,
                'subsample': 0.8,
                'colsample_bytree': 0.8,
                'random_state': self.seed + fold,
                'verbosity': -1,
                'objective': 'regression',
            }

            # LightGBM GPU
            if Config.USE_GPU and torch.cuda.is_available():
                lgb_params.update({'device': 'gpu', 'gpu_platform_id': 0, 'gpu_device_id': 0})
                print("  LightGBM: GPU")
            else:
                print("  LightGBM: CPU")

            # CatBoost params
            cat_params = {
                'iterations': 5000,
                'learning_rate': 0.05,
                'depth': 8,
                'random_seed': self.seed + fold,
                'verbose': False,
                'loss_function': 'RMSE',
                'eval_metric': 'RMSE',
            }

            # CatBoost GPU
            if Config.USE_GPU and torch.cuda.is_available():
                cat_params.update({'task_type': 'GPU', 'devices': '0'})
                print("  CatBoost: GPU")
            else:
                print("  CatBoost: CPU")

            # Train XGB dx
            xgb_dx = XGBRegressor(**xgb_params)
            xgb_dx.fit(
                X_tr, ydx_tr,
                eval_set=[(X_va, ydx_va)],
                verbose=False,
                early_stopping_rounds=Config.ES_ROUNDS
            )
            self.models_dx['xgb'].append(xgb_dx)

            # Train XGB dy
            xgb_dy = XGBRegressor(**xgb_params)
            xgb_dy.fit(
                X_tr, ydy_tr,
                eval_set=[(X_va, ydy_va)],
                verbose=False,
                early_stopping_rounds=Config.ES_ROUNDS
            )
            self.models_dy['xgb'].append(xgb_dy)

            # Train LGB dx
            lgb_dx = LGBMRegressor(**lgb_params)
            lgb_dx.fit(
                X_tr, ydx_tr,
                eval_set=[(X_va, ydx_va)],
                eval_metric='rmse',
                callbacks=[]
            )
            self.models_dx['lgb'].append(lgb_dx)

            # Train LGB dy
            lgb_dy = LGBMRegressor(**lgb_params)
            lgb_dy.fit(
                X_tr, ydy_tr,
                eval_set=[(X_va, ydy_va)],
                eval_metric='rmse',
                callbacks=[]
            )
            self.models_dy['lgb'].append(lgb_dy)

            # Train Cat dx
            cat_dx = CatBoostRegressor(**cat_params)
            cat_dx.fit(
                X_tr, ydx_tr,
                eval_set=(X_va, ydx_va),
                use_best_model=True
            )
            self.models_dx['cat'].append(cat_dx)

            # Train Cat dy
            cat_dy = CatBoostRegressor(**cat_params)
            cat_dy.fit(
                X_tr, ydy_tr,
                eval_set=(X_va, ydy_va),
                use_best_model=True
            )
            self.models_dy['cat'].append(cat_dy)

            # Train NN dx
            nn_dx = self._train_neural_network(
                X_tr.values, ydx_tr, X_va.values, ydx_va, seed=self.seed + fold
            )
            self.nn_models_dx.append(nn_dx)

            # Train NN dy
            nn_dy = self._train_neural_network(
                X_tr.values, ydy_tr, X_va.values, ydy_va, seed=self.seed + fold + 100
            )
            self.nn_models_dy.append(nn_dy)

            # Cleanup
            Config.cleanup_gpu_memory()

    # Train a neural net
    def _train_neural_network(self, X_tr, y_tr, X_va, y_va, seed=42):
        # Seeds
        set_all_seeds(seed)

        # Device
        device = torch.device(Config.GPU_DEVICE)

        # Datasets
        ds_tr = TensorDataset(torch.FloatTensor(X_tr), torch.FloatTensor(y_tr.reshape(-1, 1)))
        ds_va = TensorDataset(torch.FloatTensor(X_va), torch.FloatTensor(y_va.reshape(-1, 1)))

        # Loaders
        ld_tr = DataLoader(ds_tr, batch_size=Config.NN_BATCH_SIZE, shuffle=True)
        ld_va = DataLoader(ds_va, batch_size=Config.NN_BATCH_SIZE, shuffle=False)

        # Model
        model = SimpleNN(X_tr.shape[1]).to(device)

        # Loss and optimizer
        criterion = nn.MSELoss()
        optimizer = torch.optim.Adam(model.parameters(), lr=Config.NN_LEARNING_RATE)

        # Early stopping trackers
        best_loss = float('inf')
        best_state = model.state_dict()
        patience = 0

        # Epoch loop
        for epoch in range(Config.NN_EPOCHS):
            # Train mode
            model.train()

            # Loss accumulator
            train_losses = []

            # Batches
            for xb, yb in ld_tr:
                xb = xb.to(device)
                yb = yb.to(device)
                optimizer.zero_grad()
                out = model(xb)
                loss = criterion(out, yb)
                loss.backward()
                optimizer.step()
                train_losses.append(loss.item())

            # Eval mode
            model.eval()

            # Validation loss accumulator
            val_losses = []

            # No grad
            with torch.no_grad():
                for xb, yb in ld_va:
                    xb = xb.to(device)
                    yb = yb.to(device)
                    out = model(xb)
                    loss = criterion(out, yb)
                    val_losses.append(loss.item())

            # Average validation loss
            val = float(np.mean(val_losses))

            # Early stopping update
            if val < best_loss:
                best_loss = val
                best_state = model.state_dict()
                patience = 0
            else:
                patience += 1
                if patience >= Config.NN_PATIENCE:
                    break

        # Restore best state
        model.load_state_dict(best_state)

        # Return model
        return model

    # Generate predictions for test
    def generate_predictions(self):
        # Assemble test matrix
        X_test = self.test_data[self.numerical_features + self.categorical_features].copy()

        # Transform categoricals
        X_test = self._transform_categoricals(X_test, self.categorical_features)

        # Fill NAs
        X_test = X_test.fillna(0)

        # Scale numericals
        X_test.loc[:, self.numerical_features] = self.num_scaler.transform(X_test[self.numerical_features])

        # Ensemble predictions
        dx = self._ensemble_prediction(X_test.values, self.models_dx, self.nn_models_dx)
        dy = self._ensemble_prediction(X_test.values, self.models_dy, self.nn_models_dy)

        # Predicted absolute positions
        self.test_data['predicted_x'] = self.test_data['final_pre_throw_x'] + dx
        self.test_data['predicted_y'] = self.test_data['final_pre_throw_y'] + dy

        # Constraints
        self.test_data = self._apply_constraints(self.test_data)

        # Smoothing
        self.test_data = self._smooth_trajectories(self.test_data)

        # Return predictions
        return self.test_data

    # Weighted ensemble
    def _ensemble_prediction(self, X, tree_models, nn_models):
        # Tree weights
        weights = {'xgb': 0.34, 'lgb': 0.34, 'cat': 0.22}

        # Bucket
        preds = []

        # Tree average per family
        for name, models in tree_models.items():
            if len(models) == 0:
                continue
            fold_preds = [m.predict(X) for m in models]
            avg = np.mean(fold_preds, axis=0)
            if name in weights:
                preds.append(avg * weights[name])

        # Neural nets
        if len(nn_models) > 0:
            device = torch.device(Config.GPU_DEVICE)
            X_tensor = torch.FloatTensor(X).to(device)
            nn_fold = []
            for m in nn_models:
                m.eval()
                with torch.no_grad():
                    p = m(X_tensor).cpu().numpy().squeeze()
                nn_fold.append(p)
            nn_avg = np.mean(nn_fold, axis=0)
            preds.append(nn_avg * 0.10)

        # Sum
        if len(preds) == 0:
            return np.zeros(X.shape[0])
        return np.sum(preds, axis=0)

    # Physics constraints
    def _apply_constraints(self, df):
        # Deltas
        dx = df['predicted_x'] - df['final_pre_throw_x']
        dy = df['predicted_y'] - df['final_pre_throw_y']
        disp = np.sqrt(dx ** 2 + dy ** 2)

        # Max displacement by time
        if 'time_seconds' in df.columns:
            max_disp = Config.MAX_SPEED * df['time_seconds']
            mask = disp > max_disp
            if np.any(mask):
                scale = max_disp[mask] / (disp[mask] + 1e-6)
                dx.loc[mask] = dx[mask] * scale
                dy.loc[mask] = dy[mask] * scale
                df.loc[mask, 'predicted_x'] = df.loc[mask, 'final_pre_throw_x'] + dx[mask]
                df.loc[mask, 'predicted_y'] = df.loc[mask, 'final_pre_throw_y'] + dy[mask]

        # Clip to field bounds
        df['predicted_x'] = df['predicted_x'].clip(Config.FIELD_X_MIN, Config.FIELD_X_MAX)
        df['predicted_y'] = df['predicted_y'].clip(Config.FIELD_Y_MIN, Config.FIELD_Y_MAX)

        # Return constrained
        return df

    # Gaussian smoothing within trajectories
    def _smooth_trajectories(self, df):
        # Grouping keys
        keys = ['game_id', 'play_id', 'nfl_id']

        # Iterate groups
        for k, g in df.groupby(keys):
            if len(g) <= 3:
                continue
            idx = g.index
            df.loc[idx, 'predicted_x'] = gaussian_filter1d(g['predicted_x'].values, sigma=0.5)
            df.loc[idx, 'predicted_y'] = gaussian_filter1d(g['predicted_y'].values, sigma=0.5)

        # Return smoothed
        return df

    # Submission writer
    def create_submission_file(self, output_path="submission.csv"):
        # Guard for predictions present
        required_cols = ['predicted_x', 'predicted_y']
        for c in required_cols:
            if c not in self.test_data.columns:
                raise KeyError(f"Missing required prediction column: {c}")
    
        # Prefer the competition-provided id if available
        if 'id' in self.test_data.columns:
            # Select and rename
            sub = self.test_data[['id', 'predicted_x', 'predicted_y']].rename(columns={
                'predicted_x': 'x',
                'predicted_y': 'y'
            })
        else:
            # Fall back to constructing an id that includes frame_id if present
            base_keys = ['game_id', 'play_id', 'nfl_id']
            for k in base_keys:
                if k not in self.test_data.columns:
                    raise KeyError(f"Missing key column needed to build id: {k}")
    
            # If frame_id exists, include it in the id; if not, build without it
            if 'frame_id' in self.test_data.columns:
                # Build id with frame granularity
                self.test_data['id'] = (
                    self.test_data['game_id'].astype(str) + "_" +
                    self.test_data['play_id'].astype(str) + "_" +
                    self.test_data['nfl_id'].astype(str) + "_" +
                    self.test_data['frame_id'].astype(str)
                )
            else:
                # Build id without frame granularity
                self.test_data['id'] = (
                    self.test_data['game_id'].astype(str) + "_" +
                    self.test_data['play_id'].astype(str) + "_" +
                    self.test_data['nfl_id'].astype(str)
                )
    
            # Select and rename
            sub = self.test_data[['id', 'predicted_x', 'predicted_y']].rename(columns={
                'predicted_x': 'x',
                'predicted_y': 'y'
            })
    
        # Save to disk
        sub.to_csv(output_path, index=False)
    
        # Logs
        print(f"Submission file saved to {output_path}")
        print(f"Submission shape: {sub.shape}")
    
        # Return submission
        return sub

In [ ]:
# ================================================================================
# MAIN
# ================================================================================

def main():
    # RNG seeds
    set_all_seeds(Config.SEED)

    # GPU check
    print("🔍 Checking GPU requirements...")
    check_gpu_requirements()

    # GPU info
    Config.print_gpu_info()

    # Build predictor
    predictor = EnhancedNFLPlayerMovementPredictor(
        data_dir=Config.DATA_DIR,
        seed=Config.SEED
    )

    # Train models
    print("\n🚀 Training enhanced models with GPU acceleration...")
    predictor.train_models()

    # Generate predictions
    print("\n📊 Generating predictions...")
    preds = predictor.generate_predictions()

    # Create submission
    print("\n💾 Creating submission file...")
    sub = predictor.create_submission_file("/kaggle/working/submission.csv")

    # Final log
    print("\n✅ Enhanced pipeline completed successfully!")
    print(f"📈 Final submission shape: {sub.shape}")
    print("\nFirst 5 predictions:")
    print(sub.head())

    # Cleanup
    Config.cleanup_gpu_memory()

In [ ]:
# Entry point
if __name__ == "__main__":
    try:
        main()
    except Exception as e:
        print(f"❌ Error occurred: {e}")
        import traceback
        traceback.print_exc()
        Config.cleanup_gpu_memory()